<a href="https://colab.research.google.com/github/ANITHAKISHORE/EDA/blob/main/voiceAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install transformers

In [ ]:
! pip3 install datasets

In [ ]:
! pip3 install accelerate

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "I am taking an interview session for Automatic Speech Recognition"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
# print(output)

In [ ]:
len(output[1][0])

768

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


In [ ]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)

In [ ]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


In [ ]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [ ]:
sample = '/content/common_voice_mr_27591990.wav'
result = pipe(sample)
print(result["text"])

 संत न्यानेश्वरांच्या सात्व्या जन्म स्यताब्दि निमित्त तानी सर्वप्रथम मोग्रा फुललाया कादंबरी से अभिवाचन केले उते।


In [ ]:
reference = 'संत ज्ञानेश्वरांच्या सातव्या जन्मशताब्दीनिमित्त त्यांनी सर्वप्रथम मोगरा फुलला या कादंबरीचे अभिवाचन केले होते'

In [ ]:
# Do not use this code for the wer calculation

# def calculate_wer(reference, hypothesis):
# 	ref_words = reference.split()
# 	hyp_words = hypothesis.split()
# 	# Counting the number of substitutions, deletions, and insertions
# 	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
# 	deletions = len(ref_words) - len(hyp_words)
# 	insertions = len(hyp_words) - len(ref_words)
# 	# Total number of words in the reference text
# 	total_words = len(ref_words)
# 	# Calculating the Word Error Rate (WER)
# 	wer = (substitutions + deletions + insertions) / total_words
# 	return wer

In [ ]:
print(calculate_wer(reference, result["text"]))

0.9230769230769231


In [ ]:
# Evaluation metrics for the ASR -- Automatic Speech Recognition is (WER) Word Error Rate, Character Error Rate (CER), Sentence Error Rate(SER)